# This is an Updated version of my previous public kernel <https://www.kaggle.com/kushal1506/moa-pytorch-0-01859-rankgauss-pca-nn>

## Updates -

* Implementing Feature Engineering 
* Implementing Label Smoothing

# **If U find it helpful and consider forking , please do Upvote** :)

In [1]:
import sys
sys.path.append('../input/iterativestratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA,FactorAnalysis

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.preprocessing import QuantileTransformer

In [4]:
os.listdir('../input/lish-moa')

['train_features.csv',
 'train_drug.csv',
 'test_features.csv',
 'train_targets_nonscored.csv',
 'sample_submission.csv',
 'train_targets_scored.csv']

In [5]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
train_drugs = pd.read_csv('../input/lish-moa/train_drug.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')
train_features = train_drugs.merge(train_features,on='sig_id',how='left')
#train_target_scored = train_target_scored.merge(train_drug,on='sig_id')

In [6]:
train2 = pd.read_pickle('../input/fetching-non-targets-features-with-nn/{NB}-train_nonscore_pred.pkl')
test2 = pd.read_pickle('../input/fetching-non-targets-features-with-nn/{NB}-test_nonscore_pred.pkl')

In [7]:
train = train2.merge(train_targets_scored, on='sig_id')
#train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
#test = test2[test2['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test=test2
target = train[train_targets_scored.columns]

In [8]:
train

,sig_id,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,24,D1,1.134849,0.907687,-0.416385,-0.966814,-0.254723,-1.017473,-1.364787,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,72,D1,0.119282,0.681738,0.272399,0.080113,1.205169,0.686517,0.313396,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,48,D1,0.779973,0.946463,1.425350,-0.132928,-0.006122,1.492493,0.235577,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,48,D1,-0.734910,-0.274641,-0.438509,0.759097,2.346330,-0.858153,-2.288417,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,72,D2,-0.452718,-0.477513,0.972316,0.970731,1.463427,-0.869555,-0.375501,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,72,D1,0.237856,-1.228203,0.218376,-0.365976,-0.330177,0.569243,-0.150978,...,0,0,0,0,0,0,0,0,0,0
21944,id_fffb1ceed,24,D2,0.209361,-0.022389,-0.235888,-0.796989,-0.674009,0.919312,0.735603,...,0,0,0,0,0,0,0,0,0,0
21945,id_fffb70c0c,24,D2,-1.911021,0.587228,-0.588417,1.296405,-1.002640,0.850589,-0.304313,...,0,0,0,0,0,0,0,0,0,0
21946,id_fffcb9e7c,24,D1,0.816407,0.417618,0.431631,0.300617,1.070346,-0.024189,0.048942,...,0,0,0,0,0,0,0,0,0,0


In [9]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

In [10]:
#folds = train.copy()

#mskf = MultilabelStratifiedKFold(n_splits=5)

#for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
#    folds.loc[v_idx, 'kfold'] = int(f)

#folds['kfold'] = folds['kfold'].astype(int)
#folds

In [11]:
train

,sig_id,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,24,D1,1.134849,0.907687,-0.416385,-0.966814,-0.254723,-1.017473,-1.364787,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,72,D1,0.119282,0.681738,0.272399,0.080113,1.205169,0.686517,0.313396,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,48,D1,0.779973,0.946463,1.425350,-0.132928,-0.006122,1.492493,0.235577,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,48,D1,-0.734910,-0.274641,-0.438509,0.759097,2.346330,-0.858153,-2.288417,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,72,D2,-0.452718,-0.477513,0.972316,0.970731,1.463427,-0.869555,-0.375501,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,72,D1,0.237856,-1.228203,0.218376,-0.365976,-0.330177,0.569243,-0.150978,...,0,0,0,0,0,0,0,0,0,0
21944,id_fffb1ceed,24,D2,0.209361,-0.022389,-0.235888,-0.796989,-0.674009,0.919312,0.735603,...,0,0,0,0,0,0,0,0,0,0
21945,id_fffb70c0c,24,D2,-1.911021,0.587228,-0.588417,1.296405,-1.002640,0.850589,-0.304313,...,0,0,0,0,0,0,0,0,0,0
21946,id_fffcb9e7c,24,D1,0.816407,0.417618,0.431631,0.300617,1.070346,-0.024189,0.048942,...,0,0,0,0,0,0,0,0,0,0


In [12]:
train = train_drugs.merge(train,on='sig_id')

In [13]:
train[['sig_id','drug_id']]

,sig_id,drug_id
0,id_000644bb2,b68db1d53
1,id_000779bfc,df89a8e5a
2,id_000a6266a,18bb41b2c
3,id_0015fd391,8c7f86626
4,id_001626bd3,7cbed3131
...,...,...
21943,id_fff8c2444,6c3a459be
21944,id_fffb1ceed,df1d0a5a1
21945,id_fffb70c0c,ecf3b6b74
21946,id_fffcb9e7c,8b87a7a83


In [14]:
scored = (train[['sig_id','drug_id']]).merge(train[train_targets_scored.columns],on='sig_id')
scored

,sig_id,drug_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,b68db1d53,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,df89a8e5a,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,18bb41b2c,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,8c7f86626,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,7cbed3131,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,6c3a459be,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21944,id_fffb1ceed,df1d0a5a1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21945,id_fffb70c0c,ecf3b6b74,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21946,id_fffcb9e7c,8b87a7a83,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
targets = scored.columns[2:]

In [16]:
train

,sig_id,drug_id,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,b68db1d53,24,D1,1.134849,0.907687,-0.416385,-0.966814,-0.254723,-1.017473,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,df89a8e5a,72,D1,0.119282,0.681738,0.272399,0.080113,1.205169,0.686517,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,18bb41b2c,48,D1,0.779973,0.946463,1.425350,-0.132928,-0.006122,1.492493,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,8c7f86626,48,D1,-0.734910,-0.274641,-0.438509,0.759097,2.346330,-0.858153,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,7cbed3131,72,D2,-0.452718,-0.477513,0.972316,0.970731,1.463427,-0.869555,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,6c3a459be,72,D1,0.237856,-1.228203,0.218376,-0.365976,-0.330177,0.569243,...,0,0,0,0,0,0,0,0,0,0
21944,id_fffb1ceed,df1d0a5a1,24,D2,0.209361,-0.022389,-0.235888,-0.796989,-0.674009,0.919312,...,0,0,0,0,0,0,0,0,0,0
21945,id_fffb70c0c,ecf3b6b74,24,D2,-1.911021,0.587228,-0.588417,1.296405,-1.002640,0.850589,...,0,0,0,0,0,0,0,0,0,0
21946,id_fffcb9e7c,8b87a7a83,24,D1,0.816407,0.417618,0.431631,0.300617,1.070346,-0.024189,...,0,0,0,0,0,0,0,0,0,0


In [17]:
train_targets_nonscored = train_targets_nonscored.loc[:, train_targets_nonscored.sum() != 0]


In [18]:
nonscored_target = [c for c in train[train_targets_nonscored.columns] if c != "sig_id"]


In [19]:
# from sklearn.preprocessing import QuantileTransformer

for col in (nonscored_target):

    transformer = QuantileTransformer(n_quantiles=100, random_state=0, output_distribution="normal")
    vec_len = len(train[col].values)
    vec_len_test = len(test[col].values)
    raw_vec = train[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test[col] = transformer.transform(test[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [20]:
FOLDS = 5
vc = scored.drug_id.value_counts()
vc1 = vc.loc[(vc==6)|(vc==12)|(vc==18)].index.sort_values()
vc2 = vc.loc[(vc!=6)&(vc!=12)&(vc!=18)].index.sort_values()
folds = train.copy()
# STRATIFY DRUGS 18X OR LESS
dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=34)
tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
    dd = {k:fold for k in tmp.index[idxV].values}
    dct1.update(dd)

# STRATIFY DRUGS MORE THAN 18X
skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=34)
tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
    dd = {k:fold for k in tmp.sig_id[idxV].values}
    dct2.update(dd)
                
folds['kfold'] = folds.drug_id.map(dct1)
folds.loc[folds.kfold.isna(),'kfold'] =\
    folds.loc[folds.kfold.isna(),'sig_id'].map(dct2)
folds.kfold = folds.kfold.astype('int8')

In [21]:
print(train.shape)
print(folds.shape)
print(test.shape)
print(target.shape)
print(sample_submission.shape)

(21948, 1623)
(21948, 1624)
(3624, 1416)
(21948, 207)
(3982, 207)


In [22]:
folds

,sig_id,drug_id,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,kfold
0,id_000644bb2,b68db1d53,24,D1,1.134849,0.907687,-0.416385,-0.966814,-0.254723,-1.017473,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,df89a8e5a,72,D1,0.119282,0.681738,0.272399,0.080113,1.205169,0.686517,...,0,0,0,0,0,0,0,0,0,2
2,id_000a6266a,18bb41b2c,48,D1,0.779973,0.946463,1.425350,-0.132928,-0.006122,1.492493,...,0,0,0,0,0,0,0,0,0,4
3,id_0015fd391,8c7f86626,48,D1,-0.734910,-0.274641,-0.438509,0.759097,2.346330,-0.858153,...,0,0,0,0,0,0,0,0,0,3
4,id_001626bd3,7cbed3131,72,D2,-0.452718,-0.477513,0.972316,0.970731,1.463427,-0.869555,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,6c3a459be,72,D1,0.237856,-1.228203,0.218376,-0.365976,-0.330177,0.569243,...,0,0,0,0,0,0,0,0,0,3
21944,id_fffb1ceed,df1d0a5a1,24,D2,0.209361,-0.022389,-0.235888,-0.796989,-0.674009,0.919312,...,0,0,0,0,0,0,0,0,0,2
21945,id_fffb70c0c,ecf3b6b74,24,D2,-1.911021,0.587228,-0.588417,1.296405,-1.002640,0.850589,...,0,0,0,0,0,0,0,0,0,2
21946,id_fffcb9e7c,8b87a7a83,24,D1,0.816407,0.417618,0.431631,0.300617,1.070346,-0.024189,...,0,0,0,0,0,0,0,0,0,1


In [23]:
test

,sig_id,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,vasopressin_receptor_agonist,vasopressin_receptor_antagonist,ve-cadherin_antagonist,vesicular_monoamine_transporter_inhibitor,vitamin_k_antagonist,voltage-gated_potassium_channel_activator,voltage-gated_sodium_channel_blocker,wdr5_mll_interaction_inhibitor,xanthine_oxidase_inhibitor,xiap_inhibitor
0,id_0004d9e33,24,D1,-0.780778,0.261727,-0.779608,0.640179,1.532089,-0.191112,-0.289497,...,1.447114,0.574357,-0.970025,-0.714126,0.451300,0.267061,1.445242,-0.319964,0.477219,1.034408
1,id_001897cda,72,D1,-0.245017,0.417006,1.168949,-0.708951,-0.517363,-0.390555,-2.213382,...,-0.850107,-0.027009,-0.268354,1.190610,-0.109237,-0.333392,-0.845099,-0.815416,-0.966739,-0.445398
2,id_00276f245,24,D2,0.628172,0.359716,0.366921,0.616193,-0.839624,-1.149929,0.763103,...,-0.816158,-0.546190,-0.474715,0.123671,0.318293,-0.683429,-0.690714,0.582873,-0.073398,-0.663352
3,id_0027f1083,48,D1,-0.562820,-1.449243,1.658487,0.302608,-0.840776,0.000017,0.657583,...,0.005072,-0.227554,-0.276081,-0.089392,0.721664,-0.004394,-0.190383,1.915329,1.956936,-0.204723
4,id_006fc47b8,48,D2,0.490815,0.901390,-1.024895,-2.095041,0.741717,-0.449766,-0.637682,...,0.581940,-0.113373,0.284645,-0.118514,0.739548,-0.480395,0.345941,1.452136,0.362728,0.315064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3619,id_ff7004b87,24,D1,0.599272,-0.691298,2.335275,-0.965903,1.073011,1.784340,0.093554,...,-0.009265,0.275526,2.094639,0.854456,0.023701,0.557277,-0.345915,0.272977,-0.768545,0.695176
3620,id_ff925dd0d,24,D1,-0.844395,-0.282043,2.006827,0.507512,0.643034,0.434686,-1.006733,...,0.517689,0.973825,-0.227704,-1.101135,0.148054,1.217330,0.812854,-0.334841,0.989924,0.309629
3621,id_ffb710450,72,D1,-0.563702,-0.146343,0.232721,-1.052568,0.066341,0.635259,-0.019701,...,1.648775,1.300455,0.161674,-0.183051,0.715084,0.802735,1.738626,0.771326,1.075940,1.106776
3622,id_ffbb869f2,48,D2,-1.591078,-1.924736,-0.619409,1.321092,-1.622643,-0.263479,-0.487639,...,0.737908,1.203039,-0.489271,0.120873,1.673281,0.974808,0.453575,1.465630,1.299538,-0.600397


# Dataset Classes

In [24]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    

In [25]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
#         print(inputs.shape)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds
   
    

In [26]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [27]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [28]:
class Model(nn.Module):      # <-- Update
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.25)
        self.dense2 = nn.Linear(hidden_size, hidden_size)
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.25)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

In [29]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [30]:
feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id','drug_id']]
len(feature_cols)

1418

In [31]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 12
BATCH_SIZE = 128
LEARNING_RATE = 5e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5            #<-- Update
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=2048


In [32]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-3, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.2, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    
    loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        print(f"SEED: {seed}, FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"SEED: {seed} ,FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"FOLD{fold}_.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    
    model.load_state_dict(torch.load(f"FOLD{fold}_.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions


In [33]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [34]:
# Averaging on multiple SEEDS

SEED = [940, 1513, 1269,1392,1119,1303]  #<-- Update
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = predictions


SEED: 940, FOLD: 0, EPOCH: 0, train_loss: 0.4578473486573152
SEED: 940 ,FOLD: 0, EPOCH: 0, valid_loss: 0.022539118030930266
SEED: 940, FOLD: 0, EPOCH: 1, train_loss: 0.023354905266044796
SEED: 940 ,FOLD: 0, EPOCH: 1, valid_loss: 0.019379804266945404
SEED: 940, FOLD: 0, EPOCH: 2, train_loss: 0.022029225741499577
SEED: 940 ,FOLD: 0, EPOCH: 2, valid_loss: 0.018124100697391173
SEED: 940, FOLD: 0, EPOCH: 3, train_loss: 0.0209699798969255
SEED: 940 ,FOLD: 0, EPOCH: 3, valid_loss: 0.017880052163758698
SEED: 940, FOLD: 0, EPOCH: 4, train_loss: 0.02017164224947708
SEED: 940 ,FOLD: 0, EPOCH: 4, valid_loss: 0.017672560150351596
SEED: 940, FOLD: 0, EPOCH: 5, train_loss: 0.01997193627977285
SEED: 940 ,FOLD: 0, EPOCH: 5, valid_loss: 0.01775230494711329
SEED: 940, FOLD: 0, EPOCH: 6, train_loss: 0.019662799421643864
SEED: 940 ,FOLD: 0, EPOCH: 6, valid_loss: 0.017502821707988486
SEED: 940, FOLD: 0, EPOCH: 7, train_loss: 0.01926733624945948
SEED: 940 ,FOLD: 0, EPOCH: 7, valid_loss: 0.017268241027041394


In [35]:
len(target_cols)


206

In [36]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)


y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0
for i in range(len(target_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)
    

CV log_loss:  0.015471304904505647


In [37]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.to_csv('submission.csv', index=False)

In [38]:
train.to_csv('oofusingnonscored.csv',index=False)
test.to_csv('testusingnonscored.csv',index=False)

In [39]:
train = train[train_targets_scored.columns]
train.columns = [c + "_pred" if (c != 'sig_id' and c in train_targets_scored.columns) else c for c in train.columns]

In [40]:
test = test[train_targets_scored.columns]
test.columns = [c + "_pred" if (c != 'sig_id' and c in train_targets_scored.columns) else c for c in test.columns]

In [41]:
train

,sig_id,5-alpha_reductase_inhibitor_pred,11-beta-hsd1_inhibitor_pred,acat_inhibitor_pred,acetylcholine_receptor_agonist_pred,acetylcholine_receptor_antagonist_pred,acetylcholinesterase_inhibitor_pred,adenosine_receptor_agonist_pred,adenosine_receptor_antagonist_pred,adenylyl_cyclase_activator_pred,...,tropomyosin_receptor_kinase_inhibitor_pred,trpv_agonist_pred,trpv_antagonist_pred,tubulin_inhibitor_pred,tyrosine_kinase_inhibitor_pred,ubiquitin_specific_protease_inhibitor_pred,vegfr_inhibitor_pred,vitamin_b_pred,vitamin_d_receptor_agonist_pred,wnt_inhibitor_pred
0,id_000644bb2,0.001392,0.001040,0.001414,0.011462,0.032967,0.005872,0.003079,0.003789,0.000462,...,0.000852,0.000884,0.002366,0.003001,0.002057,0.000710,0.000734,0.001862,0.000634,0.004641
1,id_000779bfc,0.000694,0.000760,0.001987,0.017248,0.012180,0.005321,0.003326,0.004876,0.000686,...,0.001477,0.002293,0.002366,0.003678,0.002962,0.000670,0.001430,0.003508,0.002699,0.001903
2,id_000a6266a,0.002653,0.001502,0.000730,0.001305,0.004801,0.002714,0.001387,0.016716,0.000971,...,0.000510,0.001620,0.002586,0.001800,0.031160,0.000864,0.235630,0.001668,0.000591,0.001576
3,id_0015fd391,0.000633,0.001030,0.001918,0.008965,0.007299,0.001789,0.002166,0.002984,0.000520,...,0.000588,0.005553,0.003028,0.165880,0.001506,0.001289,0.001276,0.001398,0.000365,0.000894
4,id_001626bd3,0.000920,0.001199,0.004375,0.018777,0.006893,0.002381,0.009722,0.002862,0.004204,...,0.000692,0.001978,0.005355,0.001447,0.002653,0.001487,0.001556,0.001910,0.000594,0.001852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,0.004392,0.006109,0.001097,0.011020,0.027187,0.006063,0.001875,0.004554,0.000580,...,0.000645,0.000602,0.004263,0.000912,0.001837,0.001671,0.007118,0.001498,0.000909,0.001057
21944,id_fffb1ceed,0.003739,0.001824,0.001313,0.014403,0.029963,0.004955,0.002992,0.003012,0.000590,...,0.000967,0.001032,0.002901,0.002397,0.002084,0.001127,0.002527,0.001917,0.000944,0.001843
21945,id_fffb70c0c,0.000703,0.001156,0.002237,0.004874,0.004387,0.003002,0.002621,0.007252,0.000954,...,0.000884,0.002391,0.002361,0.000913,0.006212,0.000757,0.004719,0.001988,0.035029,0.005707
21946,id_fffcb9e7c,0.000759,0.000736,0.000921,0.002499,0.002779,0.001685,0.001162,0.002025,0.000433,...,0.000489,0.000638,0.001444,0.001370,0.002756,0.000450,0.001549,0.000684,0.000794,0.001020


In [42]:
# use nonscored target in the given file as feature
# if comment out below, use predicted nonscored target
# train = train.drop(nonscored_target, axis=1)
# train = train.merge(train_targets_nonscored, on="sig_id")
# train = train_features.merge(train_targets_scored, on='sig_id')
train = train_drugs.merge(train,on='sig_id')
train = train.merge(train_targets_scored, on='sig_id')
# train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
# test = test[test['cp_type']!='ctl_vehicle'].reset_index(drop=True)

# target = train[train_targets_scored.columns]
target = train[train_targets_scored.columns]

In [43]:
 
from sklearn.preprocessing import QuantileTransformer

scored_target_pred = [c + "_pred" for c in train_targets_scored.columns if c != 'sig_id']

for col in (scored_target_pred):

    transformer = QuantileTransformer(n_quantiles=100, random_state=0, output_distribution="normal")
    vec_len = len(train[col].values)
    vec_len_test = len(test[col].values)
    raw_vec = train[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test[col] = transformer.transform(test[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [44]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()


In [45]:
scored = (train[['sig_id','drug_id']]).merge(train[train_targets_scored.columns],on='sig_id')
scored

,sig_id,drug_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,b68db1d53,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,df89a8e5a,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,18bb41b2c,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,8c7f86626,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,7cbed3131,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,6c3a459be,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21944,id_fffb1ceed,df1d0a5a1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21945,id_fffb70c0c,ecf3b6b74,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21946,id_fffcb9e7c,8b87a7a83,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
targets = scored.columns[2:]
FOLDS = 5
vc = scored.drug_id.value_counts()
vc1 = vc.loc[(vc==6)|(vc==12)|(vc==18)].index.sort_values()
vc2 = vc.loc[(vc!=6)&(vc!=12)&(vc!=18)].index.sort_values()
folds = train.copy()
# STRATIFY DRUGS 18X OR LESS
dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=34)
tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
    dd = {k:fold for k in tmp.index[idxV].values}
    dct1.update(dd)

# STRATIFY DRUGS MORE THAN 18X
skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=34)
tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
    dd = {k:fold for k in tmp.sig_id[idxV].values}
    dct2.update(dd)
                
folds['kfold'] = folds.drug_id.map(dct1)
folds.loc[folds.kfold.isna(),'kfold'] =\
    folds.loc[folds.kfold.isna(),'sig_id'].map(dct2)
folds.kfold = folds.kfold.astype('int8')

In [47]:
feature_cols = [c for c in folds.columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id','drug_id']]
len(feature_cols)

206

In [48]:
EPOCHS = 25
num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=1024

In [49]:
folds


,sig_id,drug_id,5-alpha_reductase_inhibitor_pred,11-beta-hsd1_inhibitor_pred,acat_inhibitor_pred,acetylcholine_receptor_agonist_pred,acetylcholine_receptor_antagonist_pred,acetylcholinesterase_inhibitor_pred,adenosine_receptor_agonist_pred,adenosine_receptor_antagonist_pred,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,kfold
0,id_000644bb2,b68db1d53,0.349658,-0.289539,-0.212696,0.450683,1.564897,0.770568,0.368542,-0.252670,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,df89a8e5a,-0.750057,-0.822922,0.542259,1.128286,-0.248416,0.525711,0.492083,0.180901,...,0,0,0,0,0,0,0,0,0,2
2,id_000a6266a,18bb41b2c,1.306851,0.367257,-1.321627,-1.712538,-0.802257,-0.546215,-0.725411,2.085946,...,0,0,0,0,0,0,0,0,0,4
3,id_0015fd391,8c7f86626,-0.878838,-0.306043,0.469973,0.075584,-0.602485,-0.840962,-0.167653,-0.595328,...,0,0,0,0,0,0,0,0,0,3
4,id_001626bd3,7cbed3131,-0.330481,-0.036358,2.140469,1.288297,-0.630480,-0.651166,2.009108,-0.650109,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,6c3a459be,1.960835,2.336008,-0.696197,0.386956,1.091223,0.858315,-0.373585,0.063355,...,0,0,0,0,0,0,0,0,0,3
21944,id_fffb1ceed,df1d0a5a1,1.757487,0.698792,-0.368456,0.817752,1.319811,0.358971,0.326443,-0.583172,...,0,0,0,0,0,0,0,0,0,2
21945,id_fffb70c0c,ecf3b6b74,-0.732679,-0.098881,0.801093,-0.583351,-0.844665,-0.451599,0.115364,0.902020,...,0,0,0,0,0,0,0,0,0,2
21946,id_fffcb9e7c,8b87a7a83,-0.632150,-0.870880,-0.979785,-1.056301,-1.137405,-0.879063,-0.888546,-0.979919,...,0,0,0,0,0,0,0,0,0,1


In [50]:
EPOCHS = 12
num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=1024

In [51]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = (folds)
    test_ = (test)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-3, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.2, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    
    loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        print(f"SEED: {seed}, FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"SEED: {seed} ,FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"FOLD{fold}_.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    
    model.load_state_dict(torch.load(f"FOLD{fold}_.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions


In [52]:
# Averaging on multiple SEEDS

SEED = [940, 1513, 1269,1392,1119,1303]  #<-- Update
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = predictions


SEED: 940, FOLD: 0, EPOCH: 0, train_loss: 0.49031581766093557
SEED: 940 ,FOLD: 0, EPOCH: 0, valid_loss: 0.02600311948096051
SEED: 940, FOLD: 0, EPOCH: 1, train_loss: 0.02299075436009013
SEED: 940 ,FOLD: 0, EPOCH: 1, valid_loss: 0.018287576214574715
SEED: 940, FOLD: 0, EPOCH: 2, train_loss: 0.020885295288610287
SEED: 940 ,FOLD: 0, EPOCH: 2, valid_loss: 0.01777148383724339
SEED: 940, FOLD: 0, EPOCH: 3, train_loss: 0.02010719250023797
SEED: 940 ,FOLD: 0, EPOCH: 3, valid_loss: 0.017280568779610535
SEED: 940, FOLD: 0, EPOCH: 4, train_loss: 0.019714724596427834
SEED: 940 ,FOLD: 0, EPOCH: 4, valid_loss: 0.017336875802892095
SEED: 940, FOLD: 0, EPOCH: 5, train_loss: 0.019531166761357716
SEED: 940 ,FOLD: 0, EPOCH: 5, valid_loss: 0.017618483206366792
SEED: 940, FOLD: 0, EPOCH: 6, train_loss: 0.0194149155817602
SEED: 940 ,FOLD: 0, EPOCH: 6, valid_loss: 0.017356636933982372
SEED: 940, FOLD: 0, EPOCH: 7, train_loss: 0.019191598506185455
SEED: 940 ,FOLD: 0, EPOCH: 7, valid_loss: 0.017309994135490236

In [53]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)


y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0
for i in range(len(target_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)
    

CV log_loss:  0.015553535508611153


In [54]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.to_csv('submission.csv', index=False)

In [55]:
sub

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.001083,0.001579,0.002023,0.012925,0.018129,0.004193,0.002678,0.005745,0.000652,...,0.000933,0.000974,0.003065,0.002025,0.002121,0.001013,0.002062,0.001831,0.002111,0.001576
1,id_001897cda,0.000635,0.001336,0.002204,0.002579,0.001481,0.001421,0.002889,0.012138,0.003775,...,0.000941,0.000855,0.007578,0.000502,0.010684,0.000672,0.003614,0.000716,0.001890,0.002100
2,id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,id_00276f245,0.000861,0.001312,0.001474,0.013610,0.022418,0.004707,0.002504,0.004431,0.000554,...,0.000859,0.001680,0.002725,0.003706,0.006852,0.000819,0.002658,0.002578,0.000781,0.001889
4,id_0027f1083,0.001350,0.001637,0.002111,0.016125,0.024800,0.004374,0.004771,0.003089,0.000576,...,0.001138,0.001011,0.003443,0.002001,0.002062,0.000929,0.002013,0.002636,0.000609,0.001786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,id_ff7004b87,0.000754,0.001368,0.001186,0.003925,0.005239,0.002037,0.001441,0.002870,0.000573,...,0.000701,0.003946,0.002328,0.190618,0.006049,0.001217,0.006677,0.001199,0.000560,0.001179
3978,id_ff925dd0d,0.003056,0.001882,0.001717,0.009590,0.021868,0.005507,0.004602,0.004382,0.000860,...,0.000846,0.001138,0.003193,0.002869,0.002957,0.000970,0.006051,0.002336,0.000801,0.002726
3979,id_ffb710450,0.001527,0.001531,0.001800,0.015496,0.030998,0.005506,0.003876,0.004155,0.000624,...,0.000827,0.001048,0.003144,0.003295,0.002274,0.000894,0.002089,0.001761,0.000863,0.001684
3980,id_ffbb869f2,0.001877,0.001319,0.001285,0.020459,0.025105,0.004735,0.007660,0.002695,0.000639,...,0.000832,0.000898,0.002462,0.001488,0.002138,0.000672,0.001535,0.002292,0.000755,0.002632
